In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import networkit
import networkx

edgeFile = "wikispeedia_paths-and-graph/links.tsv"
categoryFile = "wikispeedia_paths-and-graph/categories.tsv"

## Load node data
Read data from file into Pandas dataframe. This tells us how many nodes there are. The categories are not used for community detection, but we'll use them later to inspect the communities.

In [ ]:
nodeData = pd.read_csv(
    categoryFile,
    names = ["article", "category"],
    skiprows = 13,
    delimiter = '\t'
)
nodeIds = {row[1].article:row[0] for row in nodeData.iterrows()}
print(len(nodeData), "nodes")

## Load link data
For this demo, let's treat the links as undirected. An alternative would be to only include bidirectional links, or find a community detection method that uses edge direction.

In [ ]:
graph = networkit.graph.Graph(len(nodeData))
nxGraph = networkx.Graph()
with open(edgeFile) as f:
    for line in f:
        if not line.startswith('#') and line.strip():
            splitLine = line.strip().split("\t")
            sourceId = nodeIds.get(splitLine[0], None)
            targetId = nodeIds.get(splitLine[1], None)
            if sourceId != None and targetId != None:
                graph.addEdge(sourceId, targetId)
                nxGraph.add_edge(sourceId, targetId)
print(len(graph.edges()), "edges")

## Connected components
List the sizes of the 10 largest weakly connected components.

In [ ]:
componentSizes = list(networkit.components.ConnectedComponents(graph)
                  .run()
                  .getComponentSizes()
                  .values())
componentSizes.sort(key = lambda x: -x)
componentSizes[:10]

## Community detection
Run community detection using the Parallel Louvain Method (PLM). This outputs a partition object.

In [ ]:
plmPartition = networkit.community.PLM(graph).run().getPartition()

In [ ]:
# Make a dictionary containing the ids and sizes of the non-singleton communities.
subsetSizes = {k:v for k,v in plmPartition.subsetSizeMap().items() if v > 1}
print("%d communities, of which %d have more than one member" % (
        len(plmPartition.getSubsetIds()),
        len(subsetSizes)
    ))
subsetSizes

## Inspect the communities
To help us make sense of the communities, let's plot the top article categories for each community.

In [ ]:
nodeData["community"] = plmPartition.getVector()
nodeData["parent_category"] = [_.split(".")[1] for _ in nodeData.category.values]
nodeData["child_category"] = [
    _.split(".")[2]
    if len(_.split(".")) > 2 else  _.split(".")[1]
    for _ in nodeData.category.values
]
nodeData[["article", "parent_category", "child_category", "community"]].tail()

In [ ]:
print(len(nodeData.parent_category.unique()), "parent categories")
nodeData.parent_category.unique()

In [ ]:
print(len(nodeData.child_category.unique()), "child categories")

In [ ]:
%matplotlib inline
def get_top_communities(
        community = None,
        column = "parent_category",
        top_n = 10
    ):
    if community is not None:
        plotData = nodeData[nodeData.community == community]
    else:
        plotData = nodeData
    return (plotData
     .groupby(column)
     .count()
     .sort("article", ascending=False)
     .article
     .head(top_n))
    
def plot_categories(
        community = None,
        column = "parent_category",
        top_n = 10,
        axes = None):
    '''
    Plot the frequencies of the top n categories in the specified column.
    Optionally subset the data by community'''
    
    if axes == None:
        axes = plt.axes()
    get_top_communities(community, column, top_n).plot(kind="bar", ax=axes)
    plt.ylabel("number of articles")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
plot_categories(top_n=15, column="parent_category", axes=ax1)
plot_categories(top_n=15, column="child_category", axes=ax2)

In [ ]:
for community,size in sorted(list(subsetSizes.items()), key = lambda x: -x[1]):
    print("\nCommunity %d: %d articles" % (community, size))
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    plot_categories(community, top_n=15, column="parent_category", axes=ax1)
    plot_categories(community, top_n=15, column="child_category", axes=ax2)
    plt.show()

Compare the top categories in two communities:

In [ ]:
for community in [8, 3]:
    print("\nCommunity %d" % community)
    for column in ["parent_category", "child_category"]:
        print("Top 5 %s:" % column)
        for item in get_top_communities(community, column, top_n=5).index:
            print("\t" + item)

In [ ]:
for community in [7, 9]:
    print("\nCommunity %d" % community)
    for column in ["parent_category", "child_category"]:
        print("Top 5 %s:" % column)
        for item in get_top_communities(community, column, top_n=5).index:
            print("\t" + item)

## Export graph to file
Save the graph data, including the communities we found, in a format that can be read with Gephi for visualisation.

In [ ]:
# Add node data to the NetworkX version of the graph
for index,data in nodeData.iterrows():
    nxGraph.node[index] = data
    
# Find the largest connected component
largestCC = nxGraph.subgraph(
    networkit.components.ConnectedComponents(graph)
    .run()
    .getPartition()
    .getMembers(1)
)

# Write the graph data to a file
networkx.write_graphml(largestCC, "wiki.graphml")